## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-21-01-11-27 +0000,nyt,Jair Bolsonaro Planned to Seek Asylum in Argen...,https://www.nytimes.com/2025/08/20/world/ameri...
1,2025-08-21-01-07-55 +0000,missionlocal,Dilapidated Bayview building to become S.F. ho...,https://missionlocal.org/2025/08/dilapidated-b...
2,2025-08-21-01-00-36 +0000,nyt,A Huge Cave in Kentucky Was a Smelly Disaster....,https://www.nytimes.com/2025/08/20/climate/hor...
3,2025-08-21-01-00-21 +0000,nyt,Dusky Gopher Frogs Are Making a Comeback in Mi...,https://www.nytimes.com/2025/08/20/climate/mis...
4,2025-08-21-01-00-19 +0000,nyt,"Why Shop? In Maine, the Library of Things Has ...",https://www.nytimes.com/2025/08/20/climate/mai...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,40
311,new,19
404,up,12
272,he,12
128,fire,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
103,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...,102
52,2025-08-20-22-22-41 +0000,nyt,Trump Calls for Fed Governor Lisa Cook to Resi...,https://www.nytimes.com/2025/08/20/us/politics...,87
18,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...,84
70,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...,84
25,2025-08-21-00-03-43 +0000,bbc,Fed governor says she won't be 'bullied' as Tr...,https://www.bbc.com/news/articles/cx276gg2gwro...,82


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
103,102,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...
52,47,2025-08-20-22-22-41 +0000,nyt,Trump Calls for Fed Governor Lisa Cook to Resi...,https://www.nytimes.com/2025/08/20/us/politics...
71,46,2025-08-20-21-44-31 +0000,nypost,$7.7B Second Avenue subway expansion will put ...,https://nypost.com/2025/08/20/us-news/7-7b-sec...
323,46,2025-08-20-03-12-54 +0000,nyt,He Tried to Endorse From the Pulpit. He Wound ...,https://www.nytimes.com/2025/08/19/us/politics...
220,45,2025-08-20-14-01-00 +0000,wsj,Billionaire Bill Ackman has a new fascination:...,https://www.wsj.com/us-news/education/bill-ack...
18,44,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...
93,43,2025-08-20-20-57-46 +0000,nypost,"JD Vance, Stephen Miller mock ‘old, white’ pro...",https://nypost.com/2025/08/20/us-news/jd-vance...
128,38,2025-08-20-19-42-00 +0000,nypost,Obama says Newsom’s Calif. redistricting plan ...,https://nypost.com/2025/08/20/us-news/obama-sa...
55,37,2025-08-20-22-21-26 +0000,nyt,Ousted Fire Chief in L.A. Claims Mayor Bass Re...,https://www.nytimes.com/2025/08/20/us/los-ange...
61,36,2025-08-20-22-04-47 +0000,nypost,Sen. Klobuchar sets record straight: She never...,https://nypost.com/2025/08/20/us-news/klobucha...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
